In [1]:
import numpy as np
import pandas as pd

### Задание 1

In [2]:
class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        speaker = self.speaker().T
        return rownorm(speaker * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses']
    states = ['r1', 'r2']
    lex = pd.DataFrame([
        [0.0, 1.0],
        [1.0, 1.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.5, 0.5], costs=[0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,costs
hat,0.0,1.0,0.0
glasses,1.0,1.0,0.0
prior,0.5,0.5,
alpha,1.0,,



Literal listener


,r1,r2
hat,0.0,1.0
glasses,0.5,0.5



Pragmatic speaker


,hat,glasses
r1,0.000000,1.000000
r2,0.666667,0.333333



Pragmatic listener


,r1,r2
hat,0.00,1.00
glasses,0.75,0.25


### Задание 2

#### 2.1.
Скалярная импликатура из таблицы *прагматического слушающего* из условия: усы есть только у одного, шляпа у двух (один с усами, другой - без) --> усы информативнее, значит, если говорят "шляпа", то скорее имеют в виду того, кто **без** усов (то есть r1, а не r2 (у r3 вообще нет шляпы, поэтому он отбрасывается)). То есть, "шляпа" --> НЕ "усы" (и мы видим, что у r1 вероятность больше, чем у r2 (0.75 > 0.25)).

#### 2.2.
Теперь при высказывании "шляпа" более вероятно r2, чем r1 (0.330579	< 0.669421) в отличие от предыдущего раза, где было наоборот. Объясняется тем, что ситуация r1 стала просто сильно маловероятна, чем r2 (0.1 < 0.45).

In [3]:
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 0.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0, 0, 0])
display_reference_game(basic_mod)
print("\nБуквальный слушающий:")
display(basic_mod.literal_listener())

print("\nПрагматический говорящий:")
display(basic_mod.speaker())

print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.0,1.0,0.0,0
очки,0.0,0.0,1.0,0
усы,0.0,1.0,0.0,0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Буквальный слушающий:


,r1,r2,r3
шляпа,0.181818,0.818182,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0



Прагматический говорящий:


,шляпа,очки,усы
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Прагматический слушающий:


,r1,r2,r3
шляпа,0.330579,0.669421,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0


### Задание 3

#### 3.1.
Ситуация, в которой шляпа есть у r1 и r2, очки у r2 и r3, а усы только у r3.

Получается две импликатуры: шляпа > очки (для r1 более информативна шляпа, потому что ее больше ни у кого нет, и поэтому для r2 тогда скорее скажут очки) и очки > усы (для r3 более иформативны усы, поэтому для r2 скорее скажут очки), т. е. "шляпа" --> НЕ "очки", "очки" --> НЕ "усы".

In [9]:
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 0.0, 1.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0])
display_reference_game(basic_mod)
print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,1.0,0.0,0
очки,0.000000,1.0,1.0,0
усы,0.000000,0.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Прагматический слушающий:


,r1,r2,r3
шляпа,0.666667,0.333333,0.0
очки,0.000000,0.600000,0.4
усы,0.000000,0.000000,1.0


#### 3.1.1. Цена высказывания

Поменяем цену высказывания, например, у очков.
Тогда импликатуры для "очков" (очки --> НЕ усы) не получается, т.к. "очки" вообще вряд ли будут сказаны (они сильно дороже стоят, чем все остальные высказывания).

In [17]:
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 0.0, 1.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 6, 0])
display_reference_game(basic_mod)
print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,1.0,0.0,0
очки,0.000000,1.0,1.0,6
усы,0.000000,0.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Прагматический слушающий:


,r1,r2,r3
шляпа,0.997533,0.002467,0.000000
очки,0.000000,0.500617,0.499383
усы,0.000000,0.000000,1.000000


#### 3.1.2. Исходная вероятность

Сделаем r1 маловероятным, а r2 и r3 равновероятными.
Видим, что теперь при "шляпе" выбор в пользу r2, а не r1, потому что r1 менее вероятен.

Вероятности для очков тоже немного поменялись, r2 совсем немного более вероятен чем r3. Тут небольшой конфликт: для r3 информативнее усы, а для r2 теперь более информативна шляпа, поэтому в случае если говорят "очки", то почти непонятно, кого выбрать - r2 или r3 (но т.к. вероятность r2 при шляпе все же меньше, чем вероятность r3 при усах, то r2 при очках чуть более вероятен, т.к. шляпа для r2 менее информативна, чем усы для r3).

In [10]:
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 0.0, 1.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0, 0, 0])
display_reference_game(basic_mod)
print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.0,1.0,0.0,0
очки,0.0,1.0,1.0,0
усы,0.0,0.0,1.0,0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Прагматический слушающий:


,r1,r2,r3
шляпа,0.263636,0.736364,0.000000
очки,0.000000,0.532258,0.467742
усы,0.000000,0.000000,1.000000


#### 3.1.3. Оптимальность

Видим, что импликатура в "очках" (очки --> НЕ усы) значительно усилилась.
Почему ничего не поменялось в шляпе? Можно подумать, что в случае с очками и усами, усы - маркер только r3, в то время как в случае со шляпой и очками, очки - маркер не только r2. Поэтому импликатура шляпа --> НЕ очки не усиливается.
Но с другой стороны, мы видим, что если говорится "очки", то это почти со 100% вероятностью r2, т.е. очки в некотором смысле являются маркером r2 (но не в том же смысле, что и для r3), поэтому мне не до конца понятно, почему тут не поменялось ничего в вероятностях.

In [14]:
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 0.0, 1.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=8)
display_reference_game(basic_mod)
print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,1.0,0.0,0
очки,0.000000,1.0,1.0,0
усы,0.000000,0.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,8.000000,,,



Прагматический слушающий:


,r1,r2,r3
шляпа,0.666667,0.333333,0.000000
очки,0.000000,0.992278,0.007722
усы,0.000000,0.000000,1.000000


#### 3.2.
Ситуация, где усы есть у всех, очки у r1 и r2, а шляпа только у r1.

Тут у нас получаются такие импликатуры: шляпа информативнее усов и очков (потому что она только у r1), а очки информативнее усов (потому что очков нет у r3). Это можно записать в одну "иерархию": шляпа > очки > усы.

In [18]:
lex = pd.DataFrame([
    [1.0, 0.0, 0.0],
    [1.0, 1.0, 0.0],
    [1.0, 1.0, 1.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0])
display_reference_game(basic_mod)
print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,0.0,0.0,0
очки,1.000000,1.0,0.0,0
усы,1.000000,1.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Прагматический слушающий:


,r1,r2,r3
шляпа,1.000000,0.000000,0.000000
очки,0.312500,0.687500,0.000000
усы,0.114943,0.252874,0.632184


#### 3.2.1. Цена высказывания

Если "очки" стоят сильно больше, чем остальные, то это высказывание вряд ли будет сказано, из-за чего импликатура очки --> НЕ шляпа не выводится.

In [19]:
lex = pd.DataFrame([
    [1.0, 0.0, 0.0],
    [1.0, 1.0, 0.0],
    [1.0, 1.0, 1.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 6, 0])
display_reference_game(basic_mod)
print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,0.0,0.0,0
очки,1.000000,1.0,0.0,6
усы,1.000000,1.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Прагматический слушающий:


,r1,r2,r3
шляпа,1.000000,0.000000,0.000000
очки,0.498766,0.501234,0.000000
усы,0.001636,0.001644,0.996719


3.2.2. Исходная вероятность

Если вероятность r1 маленькая, то импликатуры усиливаются, особенно очки --> НЕ шляпа.

In [21]:
lex = pd.DataFrame([
    [1.0, 0.0, 0.0],
    [1.0, 1.0, 0.0],
    [1.0, 1.0, 1.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0, 0, 0])
display_reference_game(basic_mod)
print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.0,0.0,0.0,0
очки,1.0,1.0,0.0,0
усы,1.0,1.0,1.0,0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Прагматический слушающий:


,r1,r2,r3
шляпа,1.000000,0.000000,0.00000
очки,0.046582,0.953418,0.00000
усы,0.012634,0.258596,0.72877


#### 3.2.3. Оптимальность

Повысив оптимальность, мы существенно усилили импликатуры.

In [22]:
lex = pd.DataFrame([
    [1.0, 0.0, 0.0],
    [1.0, 1.0, 0.0],
    [1.0, 1.0, 1.0]], index=['шляпа', 'очки', 'усы'], columns=['r1', 'r2', 'r3'])

basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0, 0, 0], alpha=8)
display_reference_game(basic_mod)
print("\nПрагматический слушающий:")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,0.0,0.0,0
очки,1.000000,1.0,0.0,0
усы,1.000000,1.0,1.0,0
prior,0.333333,0.333333,0.333333,
alpha,8.000000,,,



Прагматический слушающий:


,r1,r2,r3
шляпа,1.000000,0.000000,0.000000
очки,0.004026,0.995974,0.000000
усы,0.000146,0.036189,0.963665


### Задание 4

Может, можно применить это к разрешению проблемы омонимичных форм слов? Например, вид "братьев" имеют формы как аккузатива мн.ч., так и генитива. Вероятно "шляпами, очками и усами" будет контекст, а r1 и r2 - аккузатив и генитив (конкретно для этого примера). Тогда можно попытаться посчитать, какой из падежей более вероятен для этого контекста.


Но тут, разумеется, много проблем с контекстом, потому что первое слово слева не всегда показательно, а если брать более широкий левый контекст, получится слишком дробно как будто бы